# Projet ChatBot 2020
## MOREL Louis - MONET Anaïs - SDIA - TPS

In [105]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn import model_selection
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from spacy.lang.fr import French
import os

In [37]:
database_path = "/Users/monetanais/Documents/Cours/3A/chatbot/data/"

In [38]:
df = pd.DataFrame()
for filename in os.listdir(database_path):
    if not filename.startswith('.'):
        df = df.append(pd.read_excel(database_path + filename, names = ['attente','message']), ignore_index = True)

In [39]:
df = df.drop_duplicates()

In [40]:
df

,attente,message
0,impression,"Bonjour, je voudrais imprimer le document toto..."
1,impression,Je voudrais imprimer le document doc1 qui cont...
2,impression,"Bonjour, imprime moi doc2 qui contient 4500 pages"
3,impression,Impression doc3 4500 pages
4,impression,tirage mon document doc4 à 1800 pages
...,...,...
394,autre,pouvez-vous me dire pourquoi l'école est fermée ?
395,autre,5 5 5 doc documents pages
396,autre,pages doc impression 23 tata
397,autre,"Imprimes encore 1 fois mon document, je ne me ..."


## Apprentissage de l'attente
Pour pouvoir créer notre outil de classification, nous allons séparer nos données en deux parties : 
- **`X`** est utilisé pour les données sources utilisées pour l'apprentissage, c'est à dire l'ensemble des traits ici les messages reçus
- **`Y`** est utilisé pour ce que l'on cherche à prédire : impression ou autre
- **`train`** données d'apprentissage 
- **`test`** données d'évaluation

In [53]:
X = df['message']
X = pd.DataFrame(X)
y = df['attente']
y = pd.DataFrame(y)

In [54]:
print(X.head())
print(y.head())

                                             message
0  Bonjour, je voudrais imprimer le document toto...
1  Je voudrais imprimer le document doc1 qui cont...
2  Bonjour, imprime moi doc2 qui contient 4500 pages
3                         Impression doc3 4500 pages
4              tirage mon document doc4 à 1800 pages
      attente
0  impression
1  impression
2  impression
3  impression
4  impression


# Séparation des données

Pour réaliser nos premières expériences, nous allons découper automatiquement les données en jeu d'apprentissage et de test. Ici, 80% des données seront utilisées pour l'apprentissage, 20% pour les tests :

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Traitement de la colonne **`message`**. Transformation du **`message`** en sac de mots.
Nous allons dans un premier temps définir deux classes qui nous seront utiles dans les pipelines : la classe `SingleColumnSelector` qui nous permet de sélectionner une colonne de la table de données d'entrée à partir de son nom, et la classe `MultiColumnSelector` qui permet de sélectionner plusieurs colonnes de la table de données d'entrée.

In [56]:
class SingleColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, data_dict):
        return data_dict[self.key]
    
class MultiColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns].to_dict('records')
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

# Tokénisation
On va transformer le texte contenu dans message en vecteur. On va découper selon les espaces entre les mots.

In [61]:
# Fonction de tokénisation
def tokenize(text):
    return text.split(' ')

# Objet CountVectorizer pour la transformation en sac de mots
var_vectorizer = CountVectorizer(tokenizer = tokenize, min_df = 0.001)
    
# Pipeline spécifique
message_pipeline = make_pipeline(
    # Sélection de la colonne "message"
    SingleColumnSelector(key = 'message'),
    # Transformation sac de mots
    var_vectorizer
)

# Exemple d'application de la pipeline
# Apprentissage du vocabulaire à l'aide de "fit"
message_pipeline.fit(X_train)
# Transformation en sac de mots à l'aide de "transform"
res = message_pipeline.transform(X_test.head())
print("Input")
print(X_test.message.head())
print()
print("Output     Occurences")
print(res)
print()

# Affichage des traits extraits par la pipeline
feat_names = var_vectorizer.get_feature_names()
for i in range(20) :
    print(i, feat_names[i])

Input
266    merci de débuter l'impression du document 8 qu...
326    j'ai besoin d'imprimer 2 fois le doc 1 et ses ...
367    Bonjour j'aurais besoin d'imprimer doc8 faisan...
318                      tirage : 78 pages, document 34 
375                   il fait vraiment beau en ce moment
Name: message, dtype: object

Output     Occurences
  (0, 44)	1
  (0, 74)	1
  (0, 182)	1
  (0, 219)	1
  (0, 230)	1
  (0, 232)	1
  (0, 264)	1
  (0, 320)	1
  (0, 354)	1
  (0, 388)	1
  (0, 452)	1
  (1, 5)	1
  (1, 30)	1
  (1, 33)	1
  (1, 116)	1
  (1, 177)	1
  (1, 199)	1
  (1, 256)	1
  (1, 273)	1
  (1, 308)	1
  (1, 332)	1
  (1, 388)	1
  (1, 483)	1
  (2, 49)	1
  (2, 116)	1
  (2, 122)	1
  (2, 177)	1
  (2, 209)	1
  (2, 263)	1
  (2, 354)	1
  (2, 388)	1
  (3, 0)	1
  (3, 42)	1
  (3, 69)	1
  (3, 87)	1
  (3, 219)	1
  (3, 389)	1
  (3, 512)	1
  (4, 135)	1
  (4, 246)	1
  (4, 264)	1
  (4, 287)	1
  (4, 547)	1

0 
1 !
2 (doc2)
3 ,
4 ,234
5 1
6 100
7 12
8 122
9 123
10 1230
11 1234
12 124
13 127
14 128
15 13
16 132
17 13

# Lemmatisation

In [62]:
import nltk 
from nltk.corpus import stopwords

In [73]:
nlp = French()
def split_into_lemmas_spacy(desc) :
    doc = nlp(desc)
    return [w.lemma_ for w in doc]

nltk_stopwords = stopwords.words('french')+list(string.punctuation)
#print(nltk_stopwords)

In [85]:
# Objet TfidfVectorizer
msg_vectorizer = TfidfVectorizer(tokenizer=split_into_lemmas_spacy, 
                                lowercase=True, 
                                stop_words=nltk_stopwords, 
                                min_df=0.001)

# Pipeline spécifique
msg_pipeline = make_pipeline(
    SingleColumnSelector(key="message"),
    msg_vectorizer
)

# Exemple d'application de la pipeline
msg_pipeline.fit(X_train)
res = msg_pipeline.transform(X_test.head())
print("Input")
print(X_test.message.iloc[0])
print()
print("Output")
print(res)
print()

# Affichage des traits extraits par la pipeline
feat_names = msg_vectorizer.get_feature_names()
print(313, feat_names[313])
print(296, feat_names[296])
print(276, feat_names[276])
print(253, feat_names[253])
print(230, feat_names[230])

Input
merci de débuter l'impression du document 8 qui fait 345 pages

Output
  (0, 313)	0.127438375036568
  (0, 296)	0.36242349413885805
  (0, 276)	0.30952095214039715
  (0, 253)	0.22624765224992763
  (0, 230)	0.27453485130570826
  (0, 201)	0.4889483553921806
  (0, 191)	0.15858889590029673
  (0, 67)	0.35602898833656654
  (0, 41)	0.4889483553921806
  (1, 313)	0.12075898406759726
  (1, 272)	0.23328598085833516
  (1, 257)	0.21702717098664723
  (1, 238)	0.4397992545470888
  (1, 173)	0.33172742103991676
  (1, 155)	0.28996035628189465
  (1, 106)	0.3373685431707782
  (1, 30)	0.35708687908742565
  (1, 27)	0.35708687908742565
  (1, 2)	0.35708687908742565
  (2, 313)	0.12276941932202443
  (2, 296)	0.3491453960498098
  (2, 272)	0.23716980253754605
  (2, 257)	0.22064031065562548
  (2, 229)	0.4471211770776503
  (2, 182)	0.3133365352817221
  (2, 155)	0.29478771158929823
  (2, 112)	0.322094635695424
  (2, 106)	0.3429851655542987
  (2, 46)	0.3895032680258982
  (3, 409)	0.4055732295754736
  (3, 313)	0.1

/Users/monetanais/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Faire des stats sur l'occurence des mots dans message ?

In [86]:
# Union des traits
union = FeatureUnion(transformer_list = [
        ("msg_feature", msg_pipeline)
    ])

# Chaîne de prétraitement globale, composée de l'union des chaînes
preprocess_pipeline = make_pipeline(
    union
)

# Application de la chaîne à X_train
preprocess_pipeline.fit(X_train)
X_transformed = preprocess_pipeline.transform(X_train)
print(X_transformed.shape)

#Affichage du nombre de traits générés par chacune des chaines 
fnames_msg = msg_pipeline.named_steps['tfidfvectorizer'].get_feature_names()
print('# msg features : ', len(fnames_msg))

(318, 444)
# msg features :  444


/Users/monetanais/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [123]:
# Prétaitement + apprentissage
# LogisticRegression() 0.9
# MultinomailNB() 0.8375
# DecisionTreeClassifier() 0.8375
# RandomForestClassifier() 0.8875
# Si on veut utiliser KNneighborsClassifier il faut trouver un moyen pour déterminer k
classifier_pipeline = make_pipeline(
    preprocess_pipeline,
    LogisticRegression()
)
# Apprentissage avec les données d'entraînement
classifier_pipeline.fit(X_train, y_train)
# Test sur des données issues du jeu de test (uniquement les premières lignes)
predicted = classifier_pipeline.predict(X_test.head(20))
print(X_test.head(20))
print()
print("Predicted :")
print(predicted)
print()
print("Expected : ")
print(y_test.head(20))

                                               message
266  merci de débuter l'impression du document 8 qu...
326  j'ai besoin d'imprimer 2 fois le doc 1 et ses ...
367  Bonjour j'aurais besoin d'imprimer doc8 faisan...
318                    tirage : 78 pages, document 34 
375                 il fait vraiment beau en ce moment
95   j'aimerais que tu me tires mon document doc0 q...
114       quelle est la météo de demain a Strasbourg ?
262  J'ai un document très long à imprimer. J'aimer...
42    Bonjour, tire moi les 132 pages du document doc6
159  où en est l'impression de mon document doc7 de...
239  Lances moi l'impression du doc6 qui fait 56 pa...
13         Bonjour, imprime le document toto à 3 pages
54                    tire doc8  qui contient 67 pages
258      Pourriez-vous m'imprimer le doc8 avec 3 pages
260                    J'ai besoin des 9 pages du doc1
276  commences par imprimer le document 65 et ses 8...
289                      imprimez document 8 34pages !
23        

/Users/monetanais/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [124]:
all_predictions = classifier_pipeline.predict(X_test)
print(all_predictions)

['impression' 'impression' 'impression' 'impression' 'autre' 'impression'
 'autre' 'impression' 'impression' 'impression' 'impression' 'impression'
 'impression' 'impression' 'impression' 'impression' 'autre' 'impression'
 'autre' 'impression' 'autre' 'impression' 'autre' 'autre' 'impression'
 'impression' 'impression' 'impression' 'autre' 'autre' 'impression'
 'impression' 'impression' 'impression' 'impression' 'impression' 'autre'
 'autre' 'autre' 'autre' 'impression' 'impression' 'autre' 'autre'
 'impression' 'impression' 'impression' 'autre' 'impression' 'impression'
 'impression' 'impression' 'impression' 'impression' 'impression'
 'impression' 'autre' 'impression' 'impression' 'impression' 'impression'
 'impression' 'impression' 'impression' 'impression' 'autre' 'autre'
 'autre' 'impression' 'impression' 'impression' 'autre' 'impression'
 'autre' 'autre' 'autre' 'autre' 'impression' 'impression' 'impression']


In [125]:
print('accuracy', accuracy_score(y_test, all_predictions))
labels = np.unique(y_test)
cm =  confusion_matrix(y_test, all_predictions, labels=labels)
confusion_df = pd.DataFrame(cm, index=labels, columns=labels)
print('confusion matrix\n', confusion_df)
print('(row=expected, col=predicted)')

accuracy 0.9
confusion matrix
             autre  impression
autre          23           6
impression      2          49
(row=expected, col=predicted)


In [128]:
test = pd.DataFrame({'message' : ['Quelle heure est-il ?']})
test

,message
0,Quelle heure est-il ?


In [129]:
classifier_pipeline.predict(test)

array(['autre'], dtype=object)